In [1]:

# This code was made with the help of chat-gpt

import numpy as np
from numpy.linalg import inv, pinv, norm


# Utils

In [2]:



def generate_linear_weight(input_size, hidden_layer_size_list, output_size, epsilon):
    """
    The aim of this code is to generate the weight matrix so that the weights follows the condition for epsilon
    THIS HAS NOT BEEN ACHIEVED ! the epsilon in this code is not the thoretical epsilon
    """
    nb_elements = 0
    previous_size = 0
    for next_size in hidden_layer_size_list:
        nb_elements += previous_size*next_size
        previous_size = next_size
    nb_elements += previous_size*output_size

    prob = 0.01

    sigma = np.sqrt ((epsilon*prob)/(7*nb_elements))
    # print(f"sigma is {sigma}")

    weight_list = []
    previous_size = input_size

    zeros= 0
    for next_size in hidden_layer_size_list:
        weight_list.append(np.random.random(size = (previous_size, next_size))*sigma*zeros)
        previous_size = next_size
        zeros = (zeros or 1)
        # print(f"zeros is {zeros}")
    
    weight_list.append((np.random.random(size = (previous_size, next_size))*sigma).T[0].reshape((previous_size, 1)))

    return weight_list


def generate_data(n, d, noise_variance = .1, bias=1):
    """Generates random Gaussian data with noise."""
    X = np.random.normal(0, 1, (n, d))
    Xtest = np.random.normal(0, 1, (n, d))
    w_true = np.random.normal(0, 1, (d,1))
    noise = np.random.normal(0, 1, (n, 1))
    y = X @ w_true + noise_variance * noise + bias
    noise2 = np.random.normal(0, 1, (n,1))
    ytest = Xtest @ w_true + noise_variance * noise2 + bias
    w_star = np.linalg.pinv(X.T @ X) @ X.T @ y
    return (X, y, Xtest, ytest, w_star, w_true)


def make_augmented_matrix(slope, bias):
    in_dim, out_dim = slope.shape

    # Create augmented matrix of shape (in_dim+1, out_dim+1)
    aug = np.zeros((in_dim + 1, out_dim + 1))

    # Top-left corner: 1
    aug[0, 0] = 1

    # Top row (bias)
    aug[0, 1:] = bias

    # Bottom-left: already 0
    # Bottom-right: slope
    aug[1:, 1:] = slope

    return aug




In [3]:

class AffineModel():
    def __init__(self, input_size, hidden_layer_size_list, output_size, epsilon=1):
        self.input_size= input_size

        
        hidden_layer_size_list_affine = np.copy(hidden_layer_size_list)+1

        # Initialize the weights
        weight_list_affine = generate_linear_weight(input_size+1,hidden_layer_size_list_affine, output_size+1, epsilon) 
        self.biases = []
        self.slopes = []



        for i in range(len(weight_list_affine)-1):
            # print(i)
            # print(weight_list_affine[i].shape)
            bias = weight_list_affine[i][0, 1:]
            # print(bias.shape)
            self.biases.append(bias)
            slope = weight_list_affine[i][1:, 1:]
            # print(slope.shape)
            self.slopes.append(slope)
            # print()
        
        # last slope and bias

        i+=1 
        # print(i)
        # print(weight_list_affine[i])
        bias = weight_list_affine[i][0, 0:]
        # print(bias.shape)
        self.biases.append(bias)
        slope = weight_list_affine[i][1:, 0:]
        # print(slope.shape)
        self.slopes.append(slope)
        # print()

        # print(self.biases)
        # print(self.slopes)


    def __call__(self, X):
        return self.forward(X)
    


    def step(self, X_train, y_train, lr=0.01):
        # Forward pass
        y_pred = self.forward(X_train)
        N = X_train.shape[0]

        # Compute gradient of loss w.r.t. output
        dA = (2 / N) * (y_pred - y_train)   # dL/dA_last

        # List to store gradients for each weight matrix
        bias_grad_list = [None] * len(self.biases)
        slopes_grad_list = [None] * len(self.slopes)

        # Backward pass: reverse order
        for i in reversed(range(len(bias_grad_list))):

            W = self.slopes[i]
            b = self.biases[i]

            # to do: update

            # a_{i-1} is either input X or previous partial output
            if i == 0:
                A_prev = self.lastX
            else:
                A_prev = self.partial[i-1]

            # Gradient wrt W_i:   dW = A_prev.T @ dA
            grad_W = A_prev.T @ dA
            slopes_grad_list[i] = grad_W
            grad_b = dA.sum(axis = 0)
            bias_grad_list[i] = grad_b

            # Backprop to previous layer: dA = dA @ W.T
            dA = dA @ W.T

        # Update parameters
        for i in range(len(self.biases)):
            self.biases[i] -= lr * bias_grad_list[i]
            self.slopes[i] -= lr * slopes_grad_list[i]



    
    
    def forward(self, X):

        if X.shape[1] != self.input_size:
            print("size of input do not match size of model")
            return
        
        self.lastX = X
        y = X

        self.partial = []

        for b, W in zip(self.biases, self.slopes):
            y = b + y@W
            self.partial.append(y)
        return y
    

    def get_w_prod_and_bias(self):
        """
        Computes the equivalent single-layer slope vector (W_prod)
        and bias scalar/vector (b_prod) for the whole affine network.
        """
        # Start with first layer
        W_prod = self.slopes[0].copy()
        b_prod = self.biases[0].copy()

        for i in range(1, len(self.slopes)):
            # Update the bias: propagate previous bias through next slope
            b_prod = b_prod @ self.slopes[i] + self.biases[i]
            # Update the slope: multiply slopes
            W_prod = W_prod @ self.slopes[i]

        return W_prod, b_prod






# Test Affine Model

In [4]:
    
np.random.seed(42)

# We generate 6 data points and 
# We have 4 slope parameter + 1 bias parameter = 5 parameters in total
# This means we are in the overdetermined setting: more equations than uknowns 

n = 6
d = 4


(X, y, Xtest, ytest, w_star, w_true) =  generate_data(n, d, noise_variance=10)


# We define a model with 3 hidden layers
model = AffineModel(d, [ d, d, d], 1)


In [5]:




train_error = []
train_error.append(np.mean((y-model(X))**2))




left_sing_vec_list = [[] for i in model.biases]
eigs_list =  [[] for i in model.biases]
right_sing_vec_list = [[] for i in model.biases]


lemma_1_list = [[] for i in range(len (model.biases)-1)]




nb_steps = 500000
for i in range(nb_steps):

    model.step(X, y, .001)


    if i%(nb_steps/100) == 0:

        train_error.append(np.mean((y-model(X))**2))


        # print(np.log(train_error[-1]))

        for  i in range(len(model.biases)) :

            slope = model.slopes[i]
            bias = model.biases[i]

            weight = make_augmented_matrix(slope, bias)


            if i != 0:
                lemma_1_list[i-1].append(weight@weight.T - WktWk)


            WktWk =  weight.T@weight

            svd_decomp = np.linalg.svd(weight)

            left_sing_vec = svd_decomp[0].T[0]
            eig = svd_decomp[1][0]
            right_sing_vec = svd_decomp[2][0]

            left_sing_vec_list[i].append(left_sing_vec)
            eigs_list[i].append(eig)
            right_sing_vec_list[i].append(right_sing_vec)





print ()
# print(train_error[0])
print(f"The training error made by our affine model after training is: {train_error[-1]}")


X_with_bias = np.hstack((np.ones((X.shape[0], 1)), X))
theta_star = np.linalg.inv(X_with_bias.T@X_with_bias)@X_with_bias.T@y
OLS_train_error = np.mean((X_with_bias@theta_star-y)**2)


print(f"The error made by the the best possible model is:              {OLS_train_error}")
print ()





The training error made by our affine model after training is: 46.796558334866575
The error made by the the best possible model is:              46.796558334866575



In [21]:


product_theta = np.eye(d+1)
previous_right_eig = 0

for  i in range(len(model.biases)) :
    if i+1 < len(model.biases):

        slope = model.slopes[i]
        bias = model.biases[i]
        weight = make_augmented_matrix(slope, bias)
    else:
        slope = model.slopes[i]
        
        bias = model.biases[i]
        weight = np.zeros((d+1, 1))

        weight[0] = bias
        weight[1:] = slope

    product_theta = product_theta@weight


    svd_decomp = np.linalg.svd(weight)


    print (f"matrix {i+1}")
    print(f"the norm of the bias vector is {norm(bias)}")
    print(f"value of the bias vector: {bias}")
    # print(f"highest left singular vectior: {svd_decomp[0].T[0]}") 
    if i>0:     
        print(f"     > cosine similarity with previous right singular vector: {previous_right_eig@svd_decomp[0].T[0].T}")
    print(f"highest singular vaule: {svd_decomp[1][0]}")
    if i ==d-1: 
        # print(f"only one singular value")
        pass
    
    else:  
        print(f"second singular vaule: {svd_decomp[1][1]}")
        print(f"sum of the rest of the singular values : {np.sum(svd_decomp[1][2:])}")

    # print(f"highest right singular vector: {svd_decomp[2][0]}")
    previous_right_eig = svd_decomp[2][0]
    print()
    



matrix 1
the norm of the bias vector is 0.21342896612233658
value of the bias vector: [-0.14717603 -0.06195894 -0.09350177 -0.10634683]
highest singular vaule: 1.6530041781081215
second singular vaule: 0.9867655405235685
sum of the rest of the singular values : 1.4757441689111546e-06

matrix 2
the norm of the bias vector is 0.15969017360820453
value of the bias vector: [-0.05049915 -0.07429144 -0.09345881 -0.09325782]
     > cosine similarity with previous right singular vector: -0.999576360815782
highest singular vaule: 1.6561000754340094
second singular vaule: 0.992661091962498
sum of the rest of the singular values : 0.007270260019941503

matrix 3
the norm of the bias vector is 0.05085680963015073
value of the bias vector: [-0.02386344 -0.01020399 -0.03682708 -0.02359228]
     > cosine similarity with previous right singular vector: 0.999081963709783
highest singular vaule: 1.652869120813458
second singular vaule: 0.9992543064015142
sum of the rest of the singular values : 0.0069958

In [7]:


print(f"Value of the product of all affine matrices at convergence: {product_theta.T}")
print(f"Minimum least squares solution of the equivalent problem:   {theta_star.T}")



Value of the product of all affine matrices at convergence: [[ 1.18935176 -2.6000652  -6.68353361  1.30049403 -0.65649629]]
Minimum least squares solution of the equivalent problem:   [[ 1.18935176 -2.6000652  -6.68353361  1.30049403 -0.65649629]]


In [8]:

for i in range(d-1):
    Tele_Sum = 0
    for j in range(len(lemma_1_list[i])):
        Tele_Sum+=lemma_1_list[i][j]
    print(f"value of the Frobenius norm of the sum of all matrices of the corollary of lemma 1: {np.sum(Tele_Sum**2)}" )
    
    
print(f"NB: These value have been calculated doing the telescopic sum of on the order of {nb_steps} elements, there might be an accumulation of rounding errors")




value of the Frobenius norm of the sum of all matrices of the corollary of lemma 1: 13.615732198479966
value of the Frobenius norm of the sum of all matrices of the corollary of lemma 1: 24.45958619054729
value of the Frobenius norm of the sum of all matrices of the corollary of lemma 1: 658837.2450181461
NB: These value have been calculated doing the telescopic sum of on the order of 500000 elements, there might be an accumulation of rounding errors
